# Generate Synthetic Data

Here, we we will generate synthetic mesh geodesics so that we can use them to test geodesic regression of parameterized and unparameterized surfaces.

In [9]:
# imports

import trimesh
import os
import sys
import subprocess
import torch
import numpy as np

In [10]:
#set directory paths

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()
h2_dir = os.path.join(work_dir, "H2_SurfaceMatch")
my28brains_dir = os.path.join(work_dir, "my28brains")
print(my28brains_dir)

sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
sys.path.append(h2_dir)
sys.path.append(my28brains_dir)

import H2_SurfaceMatch.H2_match  # noqa: E402
import H2_SurfaceMatch.utils.input_output  # noqa: E402
import H2_SurfaceMatch.utils.utils  # noqa: E402

import my28brains.default_config

/home/adele/code/my28brains/my28brains


# Generate Source/Target meshes

## 1. Pill Mesh

In [11]:
pill = trimesh.creation.capsule(height=1.0, radius=1.0, count=None)

In [12]:
#pill.show()

## 2 Ellipsoid Mesh

In [13]:
sphere = trimesh.creation.uv_sphere(radius=1)

# Create a scaling matrix for the semi-axes lengths
scales = np.array([2, 2, 3])
scale_matrix = np.diag(scales)

# Apply the scaling transformation to the mesh vertices
scaled_vertices = sphere.vertices.dot(scale_matrix)

# Create a new mesh with the scaled vertices
ellipsoid = trimesh.Trimesh(vertices=scaled_vertices, faces=sphere.faces)

In [14]:
#ellipsoid.show()

## 3. Sphere Mesh

In [15]:
sphere = trimesh.creation.uv_sphere(radius=1)

#sphere.show()

# Create Geodesics between sphere and ellipsoid meshes

In [16]:
vertices_source = sphere.vertices
faces_source = sphere.faces

vertices_target = ellipsoid.vertices
faces_target = ellipsoid.faces

source = [vertices_source, faces_source]
target = [vertices_target, faces_target]

gpu_id =1

device = torch.device(f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu")

In [57]:
geod, F0 = H2_SurfaceMatch.H2_match.H2MultiRes(
        source=source,
        target=target,
        a0=default_config.a0,
        a1=default_config.a1,
        b1=default_config.b1,
        c1=default_config.c1,
        d1=default_config.d1,
        a2=default_config.a2,
        resolutions=default_config.resolutions,
        start=None,
        paramlist=default_config.paramlist,
        device=device,
    )

before starting: Vertices then Faces for S then T
(2016, 3) (3780, 3)
(1892, 3) (3780, 3)

 ############ Iteration 0:
F0.shape (3780, 3)
geod.shape: (2, 2016, 3)

 ############ Iteration 1:
F0.shape (3780, 3)
geod.shape: (2, 2016, 3)
in timesteps:
geod.shape: (3, 2016, 3)

 ############ Iteration 2:
F0.shape (3780, 3)
geod.shape: (3, 2016, 3)
in timesteps:
geod.shape: (5, 2016, 3)
3 (3780, 3)


In [61]:
#vertices of geodesic
print(geod.shape)

#faces of each point in the gedesic
print(F0.shape)

(5, 2016, 3)
(3780, 3)


In [67]:
mesh1 = trimesh.Trimesh(vertices= geod[0],
                       faces=F0)

mesh2 = trimesh.Trimesh(vertices= geod[1],
                       faces=F0)
mesh3 = trimesh.Trimesh(vertices= geod[2],
                       faces=F0)
mesh4 = trimesh.Trimesh(vertices= geod[3],
                       faces=F0)
mesh5 = trimesh.Trimesh(vertices= geod[4],
                       faces=F0)

In [17]:
#mesh1.show()

In [18]:
#mesh2.show()

In [19]:
#mesh3.show()

In [20]:
#mesh4.show()

In [21]:
#mesh5.show()